In [1]:
from xbbbNN import visualize, get_xb_bb_positions, small_network_evaluation, head_on_network_evaluation

2023-12-15 17:15:00.422341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 17:15:00.543432: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 17:15:00.999217: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-15 17:15:00.999292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
import pymolviz as pmv
import numpy as np

In [3]:
from xbpy import rdutil

In [4]:
def f(point, orientation, acceptors, **kwargs):
    #print(point, orientation, [a.GetSymbol() for a in acceptors])
    #print(kwargs)
    return 0


In [5]:
pocket = rdutil.get_binding_pockets_by_ligand(rdutil.read_molecules("5nkd.pdb", proximityBonding=True)[0])[0]

In [6]:
all_detected = []
for oxygen in pocket.pocket_atoms:
    if not oxygen.GetSymbol() == "O":
        continue
    try:
        positions = get_xb_bb_positions(oxygen)
        #print(oxygen.GetIdx())
        all_detected.append(pmv.Points(positions, color = ["O", "C", "C", "H", "N", "H", "C", "H"], name = f"O{oxygen.GetIdx()}"))
    except:
        continue

pmv.Group(all_detected, name = "detected_acceptors").write("detected_acceptors.py")

In [7]:

visualize(head_on_network_evaluation, "5nkd.pdb").write("head_on_test.py")

100%|██████████| 1720/1720 [26:22<00:00,  1.09it/s]

[1.6636340348778091, 0.6125442528309708, 0.784431712551743, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9364860589857192, 1.7461952827534255, 0.9988125770634738, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4609620692586234, 0.8499248085247443, 1.762794855896788, 1.0223391667888342, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.591866127421825, 1.6260638196803596, 0.5713691392173449, 0.564394518192473, 0.6090077542497347, 0.8537395379794651, 1.0748317230441042, 1.6408426076468872, 0.2035598707991294, 0.5746948015813552, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10879222804130806, 0.05934987814419424, 0.03913459031913125, 0.1433684815888696, 1.1905658765794864, 1.1609821222588144, 1.0022937608666107, 1.2629424811680505, 0.27184399891309274, 0.926444038686041, 1.4328829980922055, 1.2952667035623506, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.38223343759865763, 0.0, 0.0, 0.11497884240664191, 0.6963607449505521, 0.208548669170304, 0.03978834385016717, 1.0618484755023216, 0.9569757904284398, 1.4031376479212228, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
visualize(small_network_evaluation, "5nkd.pdb").write("test.py")

 20%|█▉        | 338/1720 [04:27<18:55,  1.22it/s]